In [48]:
using JuMP
using HiGHS
using Random

### Setup

In [144]:
function get_funds(n::Int64, seed::Int64=30)
    generator = MersenneTwister(seed)
    data = Dict() 
    for i in 1:n
        data["fund $i"] = Dict("expected return" => rand(generator) > 0.5 ? -rand(generator) : rand(generator) )
    end
    return data
end

get_funds (generic function with 2 methods)

In [157]:
funds = get_funds(5, 100)

Dict{Any, Any} with 5 entries:
  "fund 1" => Dict("expected return"=>0.190313)
  "fund 3" => Dict("expected return"=>-0.645691)
  "fund 2" => Dict("expected return"=>-0.0671932)
  "fund 5" => Dict("expected return"=>-0.868194)
  "fund 4" => Dict("expected return"=>-0.526845)

In [158]:
__TOTAL_FUNDS__ = length(funds)

5

### Modeling

In [173]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [174]:
# Percentage of capital allocated to each fund
@variable(model, Allocation[key in keys(funds)] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["fund 1", "fund 3", "fund 2", "fund 5", "fund 4"]
And data, a 5-element Vector{VariableRef}:
 Allocation[fund 1]
 Allocation[fund 3]
 Allocation[fund 2]
 Allocation[fund 5]
 Allocation[fund 4]

In [175]:
# Total allocated to all fund can not exceed 100%
@constraint(model, sum(Allocation) == 100)

Allocation[fund 1] + Allocation[fund 3] + Allocation[fund 2] + Allocation[fund 5] + Allocation[fund 4] == 100.0

In [176]:
# No fund should receive more than 50% of available captial
@constraint(model, [fund in keys(funds)], Allocation[fund] <= 50)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["fund 1", "fund 3", "fund 2", "fund 5", "fund 4"]
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Allocation[fund 1] <= 50.0
 Allocation[fund 3] <= 50.0
 Allocation[fund 2] <= 50.0
 Allocation[fund 5] <= 50.0
 Allocation[fund 4] <= 50.0

In [177]:
#M Maximize total return
@objective(model, Max, sum( funds[fund]["expected return"] * Allocation[fund] for fund in keys(funds)))

0.19031281518127185 Allocation[fund 1] - 0.6456910432314067 Allocation[fund 3] - 0.06719317094984745 Allocation[fund 2] - 0.8681941503861808 Allocation[fund 5] - 0.5268448425879988 Allocation[fund 4]

In [178]:
print(model)

In [179]:
optimize!(model)

Presolving model
1 rows, 5 cols, 5 nonzeros
1 rows, 5 cols, 5 nonzeros
Presolve : Reductions: rows 1(-5); columns 5(-0); elements 5(-5)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -6.1559822116e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  6.1559822116e+00
HiGHS run time      :          0.00


In [180]:
objective_value(model)

6.155982211571221

### Post-processing

In [181]:
using Printf
@printf("| %10s | %10s | %10s |\n", "Fund", "Return", "Allocation")
for fund in sort([fund for fund in keys(funds)])
    @printf("| %10s | %10.4f | %10s |\n", fund, funds[fund]["expected return"], value(Allocation[fund]))
end

|       Fund |     Return | Allocation |
|     fund 1 |     0.1903 |       50.0 |
|     fund 2 |    -0.0672 |       50.0 |
|     fund 3 |    -0.6457 |        0.0 |
|     fund 4 |    -0.5268 |        0.0 |
|     fund 5 |    -0.8682 |        0.0 |
